# Example code of TrOCR
Colab 환경에서 TrOCR 모델을 사용해 손글씨를 인식해보는 예제입니다.

## 패키지 및 예제 데이터 다운로드하기
예제를 실행시키기 위해 python package들을 설치합니다. 예제로 사용할 이미지들도 다운로드 받습니다. Colab에서 실행하지 않는 경우 이 셀은 실행하지 않습니다.

In [ ]:
!wget https://raw.githubusercontent.com/mrsyee/dl_apps/main/ocr/requirements-colab.txt
!pip install -r requirements-colab.txt

# 예제 이미지 다운로드
!mkdir examples
!cd examples && wget https://github.com/mrsyee/dl_apps/raw/main/ocr/examples/Hello.png
!cd examples && wget https://github.com/mrsyee/dl_apps/raw/main/ocr/examples/sentence.png

## 패키지 불러오기

In [ ]:
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import matplotlib.pyplot as plt

## 예제 이미지 불러오기

In [ ]:
# Load image
image = Image.open("examples/Hello.png").convert("RGB")

plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("on")
plt.show()

## 사전 학습 모델 불러오기

huggingface의 transformers에 구현되어 있는 TrOCR 모델을 불러옵니다. transfomers 라이브러리에서 pre-train 된 TrOCR 모델을 제공합니다.

transfomers의 라이브러리에서 두 가지 class를 불러옵니다.
- TrOCRProcessor: 모델에 입력할 데이터의 전처리와 출력된 모델 결과물의 후처리를 수행합니다.
- VisionEncoderDecoderModel: 실제 모델이며 전처리된 입력을 받으면 결과물을 출력합니다.

In [ ]:
print("[INFO] Load pretrained TrOCRProcessor")
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
print("[INFO] Load pretrained VisionEncoderDecoderModel")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

## TrOCR 모델로 이미지에서 텍스트 추론하기

TrOCRProcessor와 VisionEncoderDecoderModel의 내부 함수를 이용해 전처리, 추론, 후처리를 수행하여 원하는 결과물을 얻습니다.

In [ ]:
# Preprocess
pixel_values = processor(images=image, return_tensors="pt").pixel_values
# Inference
token_ids = model.generate(pixel_values)
# Postprocess
text_from_image = processor.batch_decode(token_ids, skip_special_tokens=True)[0]

In [ ]:
text_from_image